In [1]:
import pandas as pd
import numpy as np
import gmaps
import requests

from config import g_key

In [22]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [2]:
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Rain inches (last 3hrs),Snow inches (last 3hrs)
0,0,Rikitea,PF,2020-05-10 17:30:30,-23.12,-134.97,75.36,73,5,9.73,clear sky,0,0
1,1,Yellowknife,CA,2020-05-10 17:27:55,62.46,-114.35,30.20,54,20,6.93,few clouds,0,0
2,2,Staryy Nadym,RU,2020-05-10 17:28:40,65.61,72.68,44.42,88,2,7.54,clear sky,0,0
3,3,Hobart,AU,2020-05-10 17:27:25,-42.88,147.33,50.00,71,40,9.17,scattered clouds,0,0
4,4,Nizhniy Bestyakh,RU,2020-05-10 17:30:30,61.96,129.91,30.20,74,99,6.71,overcast clouds,0,0


In [3]:
city_data_df.loc[city_data_df["Rain inches (last 3hrs)"] > 0].count()["City"]

0

In [4]:
city_data_df.loc[city_data_df["Snow inches (last 3hrs)"] > 0].count()["City"]

0

In [11]:
# Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the customer for the maximum temperature preference.

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [12]:
# Prompt the customer to answer if he or she would like it to be raining or not
rain_YorN = input("Do you want it to be raining? (yes/no) ")

Do you want it to be raining? (yes/no) no


In [13]:
# Prompt the customer to answer if he or she would like it to be snowing or not
snow_YorN = input("Do you want it to be snowing? (yes/no) ")

Do you want it to be snowing? (yes/no) no


In [14]:
if (rain_YorN.lower()=="no") & (snow_YorN.lower() =="no"):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & 
                                           (city_data_df["Max Temp"]<= max_temp) &
                                           (city_data_df["Rain inches (last 3hrs)"]==0) & 
                                           (city_data_df["Snow inches (last 3hrs)"]==0)]
elif (rain_YorN.lower() == "no") & (snow_YorN.lower() == "yes"):
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & 
                                           (city_data_df["Max Temp"]<= max_temp) & 
                                           (city_data_df["Snow inches (last 3hrs)"]>0)]

else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & 
                                           (city_data_df["Max Temp"]<= max_temp) & 
                                           (city_data_df["Rain inches (last 3hrs)"]>0)]
preferred_cities_df.count()["City"]

173

In [15]:
hotel_df = preferred_cities_df[["City","Country","Max Temp", "Current Weather", "Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

C:\Users\Danie\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Rikitea,PF,75.36,clear sky,-23.12,-134.97,
7,Guerrero Negro,MX,71.49,scattered clouds,27.98,-114.06,
15,Popondetta,PG,74.07,broken clouds,-8.75,148.25,
16,Kapaa,US,75.20,overcast clouds,22.08,-159.32,
17,Souillac,MU,73.40,scattered clouds,-20.52,57.52,


In [16]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# https://developers.google.com/maps/documentation/geocoding/start#reverse


In [17]:
#iterate through dataframe
for index, row in hotel_df.iterrows():
    #Get the lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add the lat and lng to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    
    # make request and get json data
    hotels = requests.get(url, params=params).json()
    #grab the first hotel from the results.. store name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping")

C:\Users\Danie\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [18]:
hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,Rikitea,PF,75.36,clear sky,-23.12,-134.97,Pension Maro'i
7,Guerrero Negro,MX,71.49,scattered clouds,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
15,Popondetta,PG,74.07,broken clouds,-8.75,148.25,Timos Transit House
16,Kapaa,US,75.20,overcast clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
17,Souillac,MU,73.40,scattered clouds,-20.52,57.52,Bils' Villa


In [19]:
# # upload new dataframe as csv
# # Create the output file (CSV).
hotel_df.to_csv("weather_data/WeatherPy_vacation.csv")
## check vacation data file


In [23]:
# Add pop-up marker for each city
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F </dd>

</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [30]:
# Filter the DataFrame for minimum and maximum temperature preferences, 
# and if the rain or snow accumulation is 0 inches or not using conditional statements. Do the following:
# Prompt the customer for the minimum temperature preference.
# Prompt the customer for the maximum temperature preference.
# Prompt the customer to answer if he or she would like it to be raining or not, 
# using input("Do you want it to be raining? (yes/no) ").
# Prompt the customer to answer if he or she would like it to be snowing or not,
# using input("Do you want it to be snowing? (yes/no) ")

In [33]:
# Add pop-up marker for each city
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))